# 0) Setup

This notebook is divided into multiple sections, covering different aspects and functionalities of the `fermioniq` package and services. You need to execute the cells in this section **0) Setup** before running anything else, but other than that the different sections are independent from one another.

Let's start by setting up the virtual environment and `fermioniq` client.

---
**Installation**

First, we'll install `fermioniq`.

In [ ]:
# Install fermioniq client package
!pip install fermioniq

Almost all tutorials use Qiskit (there is one example that uses Cirq -- the emulator works with both).

In [ ]:
# Install Qiskit
!pip install qiskit==0.45.2

We'll also import some modules we'll use throughout the tutorial.

In [ ]:
import rich  # pretty print
import nest_asyncio

# This code snippet makes the client compatible with an iPython notebook
nest_asyncio.apply()

---
**Client**

To use the Fermioniq emulator, you need an access token. This consists of two things:
- An ID
- A secret

In this tutorial, we will save both as global variables.

In [ ]:
import os

# Set access tokens as environment variables.
os.environ['FERMIONIQ_ACCESS_TOKEN_ID'] = input('Enter access token ID: ')
os.environ['FERMIONIQ_ACCESS_TOKEN_SECRET'] = input('Enter access token secret: ')

Next, we'll initialize an instance of the fermioniq client.

In [ ]:
import fermioniq

# Create an instance of the fermioniq client
client = fermioniq.Client(verbosity_level=1)

(Instead of saving the access token id and secret as global variables, they can also be given as input to the client using the 'access_token_id' and 'access_token_secret' keyword arguments.)

The verbosity level controls how much information the client gives you when you submit a job.
- ```0```: No status updates.
- ```1```: Status updates at the start and end of simulation.



# 1) Running a circuit

We'll start with creating a circuit, use it to create an emulation job, and then submit that job for running on the emulator. Currently we support circuits from Qiskit and Cirq. We'll use Qiskit for these tutorials.

In [ ]:
from qiskit import QuantumCircuit

def qiskit_bell_pair():
  circuit = QuantumCircuit(2)
  circuit.h(0)
  circuit.cx(0, 1)
  return circuit

circuit = qiskit_bell_pair()

# Create an emulator job
emulator_job = fermioniq.EmulatorJob(circuit=circuit)

# Send job and wait for the results
result = client.schedule_and_wait(emulator_job)

Running the job via ```schedule_and_wait``` will block program execution until the job is complete, in which case ```result``` will hold the results of the job. For asynchronous running, see Tutorial 4.

The results can be printed in a pretty way by using the Rich python library.

In [ ]:
rich.print(result)
rich.print(result.job_metadata)

Alternatively, you can print the raw output using the usual print command. This is best done via the {`amplitudes()`, `samples()`, `expectation_values()`, `metadata()`, `config()`} methods of the output object, by specifying the circuit number and run number (both 0 if there was only a single emulation performed).

In [ ]:
# Amplitudes
print("Amplitudes:", result.amplitudes(circuit_number=0, run_number=0))

# Samples
print("Samples:", result.samples(circuit_number=0, run_number=0))

# Expectation values
print("Expectation values:", result.expectation_values(circuit_number=0, run_number=0))

# Metadata
print("Metadata:", result.run_metadata(circuit_number=0, run_number=0))

# Config
print("Config:", result.config(circuit_number=0, run_number=0))

# Whole result
print("All output data:", result)

# 2) Config & output settings

The emulator has extensive configuration options. To get the most out of the emulator for very difficult circuits it will often be necessary to configure the emulator properly. However, for most circuits good results can be obtained by using the default options. The client can take care of this for you.

First let's setup a circuit.

In [ ]:
from qiskit import QuantumCircuit
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.cx(0, 1)
circuit.cx(1, 2)

print(circuit)

---
**Standard config**

Creating a job with `fermioniq.EmulatorJob(circuit)` will automatically generate config settings for that circuit.

To obtain these automatically generated config settings explicitly, you can use the `standard_config()` function:

In [ ]:
from fermioniq.config.defaults import standard_config

# Obtain a standard config for this circuit
config = standard_config(circuit)

# Take a look
rich.print(config)

# Give this config alongside the circuit to create an emulator job
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)

You can modify any of the configuration options (within limitations) by giving a custom config to the `EmulatorJob` constructor. A good starting point is usually the standard configuration.

The `standard_config()` function takes 4 arguments:
- `circuit`: This is the Cirq or Qiskit circuit you want to emulate.
- `effort`: This parameter can be used to control how much computational effort is put into simulating the circuit by the emulator. It should be a number between 0 and 1. The default setting is 0.1 (10% 'effort'), which should give good results on most small- to medium-sized circuits. A value of 1 will attempt a maximum effort emulation up to computational resource limitations.
- `noise`: Set to `True` if this config is for a noisy emulation. There is also a `standard_config_noisy()` function for convenience.

---
**Basic config settings**

As can be seen above, `standard_config()` creates a dictionary with the fields:
- `mode`
- `qubits`
- `grouping`
- `dmrg`
- `noise`
- `output`

The fields `mode`, `dmrg` and `grouping` will be explained in Tutorial 3. `Noise` has to do with mixed state emulation, which is the topic of Tutorial 5.

The remainder of this tutorial focuses on the fields that need to be set to specify the input and output of the emulator, which are:

- `initial_state`: Initial state of the emulator;
- `qubits`: list of qubit objects;
- `output`: dict of output options.

---
**Initial state**

This determines the initial state of the emulation.

The intial state is always a computational basis state, which can be specified by providing either:
- an integer between `0` and `2^(num_qubits) - 1`, or
- a bitstring specifying the state of every qubit individually.

When not specified, `initial_state` is assumed to be the all-zero state.

In [ ]:
from fermioniq.config.defaults import standard_config

# Obtain a standard config for this circuit
config = standard_config(circuit)

# Set initial state (integer)
config["initial_state"] = 0

# Set intial state (bitstring)
config["initial_state"] = "000"

---
**Output settings**


Perhaps the next thing you'd want to change is the output of the emulator. We support four types of output:
- **amplitudes**: You can ask for the amplitudes of specific basis states, or the amplitudes of all basis states if the number of qubits is not too large (10 qubits or less).
- **sampling**: You can ask for samples of basis states from the final state of the emulator, just like you'd get from a real quantum computer.
- **MPS**: You can ask for the actual matrix product state (MPS) that is generated by the emulator. This is essentially a compressed version of the full statevector, and is the most general output you can receive. From here you can compute expectation values, amplitudes, or do your own sampling.
- **expectation_values**: You can ask for expectation values of Qiskit `SparsePauliOp`s or Cirq `PauliSum`s or `PauliString`s. Tutorial 8 explores this functionality in more detail.

To change what output is provided, you modify the `"output"` field of the config, which has the following structure (where the values within `'<', '>'` are for you to choose):

In [ ]:
output_config = {"output": {
        "amplitudes": {
            "enabled": '<True / False>',
            "bitstrings": '<list of basis states (as strings or integers) / "all">',
        },

        "sampling": {
            "enabled": '<True / False>',
            "n_shots": '<integer>',
        },

        "mps":{
            "enabled": '<True / False>',
        },

        "expectation_values":{
            "enabled":  '<True / False>',
            "observables": '<dict of observables>'
        },
    }
}

For example, if you want to compute the amplitudes on the 011 and 010 states, and sample 1000 shots, you would do the following:

In [ ]:
output_config = {"output": {
        "amplitudes": {
            "enabled": True,
            "bitstrings": [3, 2], # alternatively: ["011", "010"]
        },

        "sampling": {
            "enabled": True,
            "n_shots": 1000,
        },
    }
}

# Update the standard config generated earlier, and print it:
config = standard_config(circuit, effort=1)
config.update(output_config)
rich.print(config)


# Make and submit the job, and print the output
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)
result = client.schedule_and_wait(emulator_job)
rich.print(result)

---
**Qubits**

Now is a good time to demonstrate the meaning of the config setting `qubits`. This setting tells the emulator how to interpret bitstrings -- i.e. what order the qubits should be taken to be in. The default is specific to the type of circuit that you are using. For Qiskit, the default ordering is the numerical order of the qubit objects. For Cirq, there is a default ordering obtained from Cirq itself (via the method `cirq.QubitOrder.DEFAULT.order_for(...)`)<br>

We can override this by setting the `qubits` parameter to a different order, e.g.

In [ ]:
qubits = circuit.qubits

config = standard_config(circuit)
config.update({"qubits": [qubits[2], qubits[1], qubits[0]]})

# Run the circuit using these config settings
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)
result = client.schedule_and_wait(emulator_job)
rich.print(result)


As you can see, the order of bits in the the sampled bitstrings changes accordingly.

---
**Output structure**

The result of `schedule_and_wait()` (or `schedule_async()` -- see Tutorial 4) is a dictionary with three keys: (A) `job_outputs`, (B) `job_metadata`, and (C) `status_code`.

(A) It is possible to submit several emulations within a single job. `job_outputs` is a list of dictionaries, where each dictionary in the list corresponds to the results of a single emulation. For each such result, the dictionary will have the entries:
- `circuit_number`: integer specifying the circuit that was emulated (only `0` if the job contains one circuit, or an integer in case of a batch job -- see Tutorial 8);
- `run_number`: integer specifying a separate run of that same circuit (e.g. it is possible to emulate a single circuit for several values of the bond dimension, which will create outputs for each run -- see Tutorial 3);
- `output`: dict containing the actual emulation output (i.e. `amplitudes`, `samples`, `mps` or `expectation_values`);
- `config`: the config used for the emulation;
- `metadata`: dict containing additional emulation metadata for this run (explained in Tutorial 3).

(B) The `job_metadata` dictionary contains:
- `gpu_used`: a boolean specifying if a GPU was used or not;
- `total_runs`: total number of emulations performed;
- `total_runtime`: total runtime of all emulations together.

(C) `status_code` is `0` when the job finished successfully, or `-1` when the job is still running; any other value means an error occurred.

# 3) Emulation settings

There are two MPS emulation routines available: TEBD and DMRG. This tutorial explains how to configure both routines.

**DMRG**

Emulates the circuit by breaking it down into segments called 'subcircuits', and for each subcircuit finds an MPS (of a given bond dimension `D`) that best approximates the state obtained after applying that subcircuit to the MPS resulting from the previous subcircuit.
- works for any type of circuit (long- and short-range gates);
- built to be very fast on a GPU.

**TEBD**

Emulates the circuit gate-by-gate, where each gate is applied to the MPS, and then the MPS is truncated back to a maximum bond dimension `max_D`.
- only works for circuits with gates that are nearest-neighbor from the perspective of the MPS (for long-range gates, SWAP gates need to be added manually);
- can be a bit more memory efficient than DMRG (allowing to push the bond dimension higher), but doesn't benefit much from GPU acceleration.
- often achieves a worse fidelity for a given bond dimension than DMRG.

**Unless there is a specific reason to use TEBD, we recommend using DRMG, especially when using a GPU backend.**

We will start by explaining the DMRG configuration settings. TEBD will be explained at the end of this tutorial. Note that TEBD can also be used to do a statevector emulation.

---
**DMRG**

We'll make a circuit that prepares the entangled state $\frac{1}{\sqrt{2}}\left(|110\rangle + |011\rangle\right)$, and see what effect various config settings have on the circuit and the state that the emulator obtains.

In [ ]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.cx(0, 1)
circuit.cx(1, 2)
circuit.cx(0, 1)

print(circuit)

Now we'll emulate it using the emulator with the default config.

In [ ]:
from fermioniq.config.defaults import standard_config

# Obtain a standard config for the circuit
config = standard_config(circuit)

# Give this config alongside the circuit to create an emulator job
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)

# Run the job and print the results
result = client.schedule_and_wait(emulator_job)
rich.print(result)

---
**Bond dimension**

First, we will investigate what effect changing the bond dimension of the emulation has. By default we run a 'dmrg'-type emulation, and so we will change the maximum bond dimension allowed for dmrg, by modifying the `"D"` parameter of the `"dmrg"` config settings. We will set it to 1, so that only product states can be represented. This should change the output of the emulation for the circuit we created above.

In [ ]:
config = standard_config(circuit)
dmrg_config = config["dmrg"]
dmrg_config["D"] = 1

# Run the circuit using these config settings
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)
rich.print(client.schedule_and_wait(emulator_job))

We obtain a final product state that is as close as we can get to the entangled state $\frac{1}{\sqrt{2}}\left(|110\rangle + |011\rangle\right)$ with a product state. In the metadata output, you can see that the fidelity of the emulation is 0.5.

Increasing the bond dimension makes the emulation more accurate. In principle, for arbitrarily deep circuits, a bond dimension that is exponential in the number of qubits is needed to emulate the circuit exactly. However, especially for not too deep circuits, it often turns out that in practice a much smaller than exponentially-large bond dimension is enough to get a very good approximation of the final state of the circuit. This is the main strength of using tensor network states to emulate quantum circuits.

---
**Grouping**

The grouping determines which qubits are placed together in a single MPS tensor. Qubits that become strongly entangled should -- ideally -- not be too far from each other in the MPS; it is important to group qubits in a smart way. The `standard_config` function creates a (usually quite optimal) grouping by doing some simple analysis of the input circuit, creating groups of size 2 for pure state emulation, and groups of size 1 for mixed state emulation.

If we want the grouping algorithm to create groups of different sizes, then we can specify the `group_size` field of the config to specifiy how large we want the groups to be.

In [ ]:
config = standard_config(circuit)

# Remove the automatically generated grouping from the config
del config["grouping"]

# Set the group size instead
config["group_size"] = 2

If both `grouping` and `group_size` are given, then an error will be raised upon validation.

It is also possible to input a manual `grouping`. Recall that if the input circuit is expected to entangle particular qubits, it makes sense to put them into the same group so that their entanglement structure can be preserved independently of the chosen bond dimension. Given that the circuit effectively generates a Bell-pair on qubits 0 and 2, it makes sense to group these together into a single group. Because qubit 1 is not entangled with qubits 0 and 2, we expect that a bond dimension of 1 should result in a fidelity 1 emulation. Let's try it out.

In [ ]:
config = standard_config(circuit)

dmrg_config = config["dmrg"]
dmrg_config["D"] = 1

qubits = circuit.qubits
config["grouping"] = [[qubits[0], qubits[2]], [qubits[1]]]

emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)
rich.print(client.schedule_and_wait(emulator_job))

This allows us to recover the entangled state $\frac{1}{\sqrt{2}}\left(|110\rangle + |011\rangle\right)$ with a 'product state', with a fidelity of 1. Note that for complicated circuits this will not always be possible, even if the final state is unentangled between groups. If qubits in different groups become entangled at some point during the circuit the emulator will not be able to recover the final state with a product state.

---
**Initial state routine (per subcircuit)**

For each subcircuit, the emulator starts with an intial guess for the MPS. There are several options here:
- `random` uses a randomly initialized MPS;
- `ket` uses the MPS from the previous subcircuit (this is the default setting);
- `lowerD` uses the MPS obtained from a (previous) lower bond dimension emulation of the same circuit.

We advise to use `ket` over `random`.

To use the `lowerD` option, a list of bond dimensions $[D_1, D_2, \dots, D_k]$ should be provided at the start of the emulation. The emulator will then emulate the same circuit for each value of the bond dimension $D_i$ given, starting from the lowest. When emulating the $j$-th subcircuit at bond dimension $D_i$, the $D_{i-1}$ MPS obtained at the $j$-th subcircuit will be used as an intial state for the $D_i$ emulation of the $j$-th subcircuit.

When running the emulator with a list of bond dimensions (whether the initial guess is set to `lowerD` or not), the output will have a `run_number` specifying which bond dimension was used. In the example below, the `run_number` will run from `0` (bond dimension 2) to `3` (bond dimension 16).

In [ ]:
config = standard_config(circuit)

dmrg_config = config["dmrg"]
dmrg_config["D"] = [2, 4,  8, 16]
dmrg_config["init_mode"] = "lowerD"

emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)
rich.print(client.schedule_and_wait(emulator_job))

---
**Convergence criterion**

Given a subcircuit, the emulator iteratively updates the tensors in the MPS one-by-one, and after each update the fidelity between the MPS and the state that it tries to approximate (given by applying the subcircuit to the MPS resulting from the previous subcircuit) gets computed.
The emulator will stop when one of the following convergence criteria has been reached:
- `target_fidelity` between the MPS and the state that it tries to approximate is reached (default is 1.0);
- the standard deviation of the last `convergence_window_size` (default is 2 * number of tensors in the MPS) fidelities computed goes below the `convergence_threshold` (default is 1e-5);
- `max_sweeps` (default is 1e4) is reached, in which case the emulator has not converged.

Each of these parameters can be adjusted by adjusting the numbers below.

In [ ]:
config = standard_config(circuit)

dmrg_config = config["dmrg"]
dmrg_config["target_fidelity"] = 1.0
dmrg_config["convergence_window_size"] = len(config["qubits"]) // config.get("group_size", 1)
dmrg_config["convergence_threshold"] = 1e-5
dmrg_config["max_sweeps"] = 1e4

---
**TEBD**

It is also possible to perform a TEBD emulation. This requires that all gates in the circuit are nearest-neighbor with respect to the qubit grouping, i.e. every gate should act on qubits that are in neighboring groups.

In particular, this means that TEBD only works with a manual `grouping`.

There are two configuration settings for TEBD: the maximum bond dimension `max_D`, and the `svd_cutoff`. After absorbing a gate into the MPS, a singular value decomposition is performed to truncate the MPS back to a maximum bond dimension. The number of singular values kept is the minimum of (i) `max_D` and (ii) the number of singular values that are more than a factor `svd_cutoff` of the largest singular value.


In [ ]:
config = standard_config(circuit)
config["mode"] = "tebd"
config["tebd"] = {"max_D": 2, 'svd_cutoff': 1e-6}
config["grouping"] = [[qubits[0]], [qubits[1], qubits[2]]]

emulator_job = fermioniq.EmulatorJob(config=config, circuit=circuit)

rich.print(client.schedule_and_wait(emulator_job))

---
**Statevector simulation**

It is also possible to do a full statevector simulation for up to 34 qubits (using Nvidia's cuStateVec). To do this simply set the config `mode` to `statevector`. 

**_Note:_** Currently only pure-state emulations are supported for statevector. Mixed-state emulations coming soon. 

In [ ]:
config = standard_config(circuit)
config["mode"] = "statevector"

emulator_job = fermioniq.EmulatorJob(config=config, circuit=circuit)

rich.print(client.schedule_and_wait(emulator_job))

---
**Emulation metadata (output)**

Recall that the result of `schedule_and_wait()` (or `schedule_async`) for each individual emulation contains a dictionary called `metadata` (see Tutorial 2). `metadata` is a dictionary that contains the following keys:
- `runtime`: the runtime of the emulation;
- `fidelity_product`: product of all fidelities, one for each approximation made (one fidelity per subcircuit for DMRG, and one fidelity per gate for TEBD) -- printed as 'estimated fidelity';
- `extrapolated_2qubit_gate_fidelity`: obtained as the `1/n`-th power of `fidelity_product`, where `n` is the number of two-qubit gates in the circuit* -- printed as 'estimated 2-qubit gate fidelity';
- `num_subcircuits`: (DMRG only) the number of subcircuits the circuit was decomposed into;
- `circuit_depth`: the depth of the circuit;
- `number_2qubit_gates`: number of two-qubit gates;
- `output_metadata`: metadata on the computation of the emulation output (`amplitudes`, `sampling`, `mps` or `expectation_values`).

*The `extrapolated_2qubit_gate_fidelity` can be thought of as a measure for how much truncation has happened per two-qubit gate due to the fact that we are approximating the circuit emulation by a finite-bond-dimension emulation.

# 4) Choosing different backends

The emulator can be run with different backends. So far, everything has been running on CPU. Where the emulator really shines is running on GPUs.

To see the available backends, call the `remote_configs()` method.

In [ ]:
# Get a list of available backends
backends = client.remote_configs()

for backend in backends:
    rich.print(backend)

The name of the desired backend should be specified when creating an `EmulatorJob` object. If no backend is specified, then the default one will be used (which currently is 'default-cpu').

For instance, to select the default GPU backend, and run the emulation on a GPU instance, we can do the following:

In [ ]:
from qiskit import QuantumCircuit

def qiskit_bell_pair():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit

# Requires the qiskit module
circuit = qiskit_bell_pair()

# Create the job, specifying the name of the backend
emulator_job_gpu = fermioniq.EmulatorJob(circuit=circuit, remote_config='gpu-12gb')

# Run the job (note that it may take longer than usual, due to some overhead involved in preparing a GPU instance)
result = client.schedule_and_wait(emulator_job_gpu)

# You should see in the result metadata that the job was GPU accelerated
rich.print(result)

There are several available GPU backends, each taking a different fraction of an H100, the smallest one being 'gpu-12gb'. In general, emulations involving bigger bond dimensions or more qubits requires more memory.

# 5) Noisy emulation

To perform a noisy emulation, you need to specify a noise model. There are two ways to do this:
1. Defining your own noise model. <font color='green'>Explained in this tutorial</font>
2. Choose a pre-configured noise model supported by the emulator. <font color='red'>Currently Unavailable</font>

---
**Defining a custom noise model**

Broadly speaking, a noise model is built in three steps:

1) Build a `NoiseModel` object <br>

2) Define `NoiseChannels` <br>
3) Specify when the `NoiseChannels` should be applied

1) A `NoiseModel` is defined for a list of qubits (Qiskit or Cirq qubit objects). <br><br>
<span style="color:orange">**Important note**:</span> These qubits should match the qubits in the circuit. Hence it is best to first create a circuit in Qiskit/Cirq.


In [ ]:
from qiskit import QuantumCircuit
from fermioniq import NoiseModel

# Create a 3-qubit circuit for which the noise model is defined (gates can be added later)
qiskit_circuit = QuantumCircuit(3)

noise_model = NoiseModel(qubits=qiskit_circuit.qubits, name="example-qiskit-noise-model", description="An example noise model for 3 qubits")

2) `NoiseChannel`s are imported from the client. Several common channels can be defined easily, and custom channels can be specified via their Kraus operators or Pauli channel descriptions.

In [ ]:
# These are all available channels
from fermioniq import DepolarizingChannel, PhaseDampingChannel, AmplitudeDampingChannel, PhaseAmplitudeDampingChannel, BitFlipChannel, PauliChannel, KrausChannel
import numpy as np

# Examples of channel instances -- see class descriptions for details on the input parameters.
d = DepolarizingChannel(p=0.3, num_qubits=2)
pd = PhaseDampingChannel(gamma_pd=(gamma_pd:=0.3))
ad = AmplitudeDampingChannel(gamma_ad=0.3, excited_state_population=0.1)
pad = PhaseAmplitudeDampingChannel(gamma_pd=0.3, gamma_ad=0.3, excited_state_population=0.1)
b = BitFlipChannel(p0=0.2, p1=0.4)

# Example of a custom Krauss channel, in this case re-implementing the phase damping channel using Kraus operators
k_0 = np.array([[1, 0], [0, np.sqrt(1-gamma_pd)]])
k_1 = np.array([[0,0], [0, np.sqrt(gamma_pd)]])
kraus_pd = KrausChannel(kraus_ops=[k_0, k_1])

# Example of a single-qubit depolarizing channel using Pauli strings.
# Note that the PauliChannel constructor takes a dictionary of Pauli strings and their *probability* of being applied.
p = 0.2
pauli_d = PauliChannel(pauli_probs={"I": 1 - 3*p/4, "X": p/4, "Y": p/4, "Z": p/4})

Dict representations of the channel can be printed if needed.

In [ ]:
rich.print(d.dict())
rich.print(pd.dict())
rich.print(ad.dict())
rich.print(pad.dict())
rich.print(b.dict())

rich.print(kraus_pd.dict())
rich.print(pauli_d.dict())

3) Add `NoiseChannels` to the `NoiseModel` by specifying the _conditions_ under which they are triggered. A condition is a combination of a gate and the qubits that that gate acts upon. <br><br>
For instance, a very simple noise model could be one with the following triggers and channels:
- Upon application of any 2-qubit gate to any pair of qubits (__trigger__), apply 2-qubit depolarizing noise to those __same__ qubits *after* the gate is applied (__noise channel__).
- Upon application of any 1-qubit gate to any qubit (__trigger__), apply single-qubit depolarizing noise to that __same__ qubit *before* the gate is applied (__noise channel__). <br><br>
<span style="color:orange">**Important note**:</span> noise should be specified for all gates (even if it is via a generic rule using `ANY`), and readout errors should always be specified for all qubits. This restriction can be dropped later by setting a config flag.

In [ ]:
from fermioniq import ANY

# Add two-qubit gate noise
noise_model.add(gate_name=ANY, gate_qubits=2, channel=d, channel_qubits="same", when="post")

# Add one-qubit gate noise
d_one_qubit = DepolarizingChannel(p=0.3, num_qubits=1)
noise_model.add(gate_name=ANY, gate_qubits=1, channel=d_one_qubit, channel_qubits="same", when="pre")

# We add trivial readout error, both the probability of misreading 0 (first number), and that of misreading 1 (second number) are 0
noise_model.add_readout_error(qubit = ANY, p0 = 0, p1 = 0)

---
**Noisy Emulation**

To run noisy emulation simply add the __NoiseModel__ to the emulator job. <br><br>
<span style="color:orange">**Important note**:</span> To get settings in <span style="color:orange">__standard_config__</span> that are sensible for mixed state emulation, set <span style="color:orange">__noise__ __=__ __True__</span>.



In [ ]:
from fermioniq.config.defaults import standard_config

# Add some gates to the circuit
qiskit_circuit.h(0)
qiskit_circuit.x(1)
qiskit_circuit.cx(0, 1)
qiskit_circuit.cx(1, 2)
qiskit_circuit.cx(0, 1)
print(qiskit_circuit)

# Create a standard configuration for a noisy emulation of the circuit
noisy_config = standard_config(qiskit_circuit, noise=True)
rich.print(noisy_config)

# Adding the noise model to the emulator job will automatically configure it for a noisy emulation
emulator_job = fermioniq.EmulatorJob(circuit=qiskit_circuit, config=noisy_config, noise_model=noise_model)

# Note that the output will be uninteresting, since we didn't add any gates to the circuit yet!
result = client.schedule_and_wait(emulator_job)
rich.print(result)

---
**Testing your __NoiseModel__**


1) We can apply a `NoiseModel` to a circuit to get a full overview of the resulting noisy circuit by using `on_circuit(circuit)`. This returns a dictionary with two keys:
- `noise_by_gate`: A list of dictionaries, one for each gate in the circuit, giving the gate and the noise channels that are applied before and after the gate.
- `missing_noise`: A list of dictionaries, one for each gate in the circuit for which no noise channels were found, giving the gate, the position in the circuit, and the trace of the lookup process (useful for debugging). <br><br>


2) We can inspect the channels __triggered__ by a specific gate on specific qubits by using `get_noise_channels(gate_name, gate_qubits, trace=False)`. By setting the `trace` flag to `True`, the function also returns the lookup trace: a dictionary describing how the noise model arrived at the channels that it did (i.e. where it did and did not find matches). This is useful for debugging.

In [ ]:
# Define a circuit
qiskit_circuit.h(0)
qiskit_circuit.cx(0, 1)
qiskit_circuit.cx(1, 0)
qiskit_circuit.h(2)
qiskit_circuit.cx(1, 2)
qubits = qiskit_circuit.qubits


# 1: Full overview of the noisy circuit
noise_effects = noise_model.on_circuit(qiskit_circuit)
rich.print("Noise effects:")
rich.print(noise_effects)

# 2: Noise for specific gate and qubits
channels, trace = noise_model.get_noise_channels(gate_name="h", gate_qubits=[qubits[0]], trace=True)

rich.print("Lookup trace for H on q_0:")
rich.print(trace)
rich.print("(matches rule 1: H on q_0)")

channels, trace = noise_model.get_noise_channels(gate_name="cx", gate_qubits=[qubits[1], qubits[2]], trace=True)
rich.print("Lookup trace for CX on q_1 and q_2:")
rich.print(trace)
rich.print("(matches rule 5: CX on any 2 qubits)")

---
**Full overview of options and behaviour**

We will now explain in more detail:

1) Which __triggers__ can be defined <br><br>
2) How a `NoiseModel` handles multiple definitions of channels __triggered__ by the same gate and qubits. <br><br>
3) Which errors might be thrown when building a `NoiseModel`.

**Triggers**

Noise models can be as complicated or as general as desired. The only restriction (currently) is that the channels act on either one or two qubits.

The `add` function takes the following arguments:
- `gate_name`: The name of the gate that triggers the noise. Can be either a gate name for a Qiskit or Cirq circuit as a string, e.g. 'cx', 'h', 'CNOT', 'H', 'Y', or `ANY` (see below);
- `gate_qubits`: The qubits acted upon by the gate that triggers the noise. Can be a particular set of qubits such as ('q1', 'q2'), 'q0', or 'q3', or an integer (1 or 2) specifying the number of qubits the gate acts on;
- `channel`: The channel that is applied. Can be any one of the fermioniq noise channels (see above);
- `channel_qubits`: The qubits that the channel acts on. Can be a particular set of qubits, an integer (1 or 2) specifying the number of qubits, or 'same' to apply the channel to the same qubits as the gate (i.e. `gate_qubits`);
- `when`: When the noise is applied. Can be either 'pre' or 'post', determining whether the channel should be applied before or after the gate.

(The `ANY` constant from `fermioniq` can be used to match to any gate or set of qubits)

Above, `gate_name` and `gate_qubits` together specify the trigger, and `channel` and `channel_qubits` specify the noise channel that should be applied before or after the gate in response to this trigger.

Multiple channels can be added for the same __trigger__, in this case the order of application is:
1. Channels in "pre", in the order in which they were added (`when='pre'`).
2. The gate itself.
3. Channels in "post", in the order in which they were added (`when='post`).


For defining __triggers__, all gate names in Cirq and Qiskit are available (depending on which type of circuit is being used).<br> <br>


For instance, suppose we want to construct a noise model with the following 'rules':

1. <span style="color:white">Whenever a `H` gate acts on qubit `q0`, apply a `phase damping channel` to `q0`, then an `amplitude damping channel` to `q1` (modelling some kind of cross-talk).</span><br><br>
2. <span style="color:white">Whenever a `CX` gate acts on qubits `q0` and `q1`, apply a `depolarizing channel` to those qubits, as well as an `amplitude damping channel` to qubit `q0` (the control qubit) _before the gate_.</span><br><br>
3. <span style="color:white">Whenever a `CX` gate acts on qubits `q1` and `q0`, apply a `depolarizing channel` to those qubits, as well as an `amplitude damping channel` to qubit `q1` (the control qubit) _before the gate_. Note that this is the same as the previous rule, but with the qubits swapped.</span><br><br>
4. <span style="color:white">Whenever a `H` gate acts on `any qubit`, apply a `phase-amplitude damping channel` to that qubit.</span><br><br>
5. <span style="color:white">Whenever `any` gate acts on `any two` qubits, apply a `depolarizing channel` to those same qubits.</span><br><br>
6. <span style="color:white">Whenever `any` gate acts on `any one or two` qubits, apply a `phase damping channel` to those same qubits.</span><br><br>
7. <span style="color:white">Qubit `q0` has readout error where 0 is flipped to 1 with probability 0.3, and 1 is flipped to 0 with probability 0.6.</span><br><br>
8. <span style="color:white">All other qubits have readout errors where 0 is flipped to 1 with probability 0.1, and 1 is flipped to 0 with probability 0.2.</span><br><br>

Note that rules 1, 2, and 3 are specific in terms of both the gate and the qubits that it acts on. Rule 4 is specific in terms of the gate, but generic in terms of the qubits. Rules 5 and 6 are generic in terms of both the gate and the qubits, with 5 specifiying a number of qubits, and 6 specifying 'any' qubits.

In [ ]:
from fermioniq import NoiseModel, ANY
import qiskit

# Standard qiskit gate constructors are given here
from qiskit.circuit.library import standard_gates
# print(dir(standard_gates))

from qiskit.circuit.library import HGate, CXGate

# These strings correspond to those used in the qiskit 'circuit.add()' function, i.e. "h", "cx"
# Alternatively, they can be obtained from instructions in a qiskit QuantumCircuit
HGate_name = HGate().name
CXGate_name = CXGate().name
assert HGate_name == "h"
assert CXGate_name == "cx"

# Let's make an empty circuit for which the noise model will be defined
qiskit_circuit = qiskit.QuantumCircuit(3)
# Qubits on which the noise model acts
qubits = list(qiskit_circuit.qubits)

noise_model = NoiseModel(qubits=qubits, name="example-qiskit-noise-model", description="An example noise model for 3 qubits")

# Rule 1; first apply phase damping, then amplitude damping
noise_model.add(gate_name="h", gate_qubits=[qubits[0]], channel=pd, channel_qubits=[qubits[0]])
noise_model.add(gate_name="h", gate_qubits=[qubits[0]], channel=ad, channel_qubits=[qubits[1]])


# Rule 2.
noise_model.add(gate_name="cx", gate_qubits=qubits[0:2], channel=d, channel_qubits=qubits[0:2])
noise_model.add(gate_name="cx", gate_qubits=qubits[0:2], channel=ad, channel_qubits=[qubits[0]], when="pre")

# # Rule 3.
noise_model.add(gate_name="cx", gate_qubits=[qubits[1], qubits[0]], channel=d, channel_qubits="same")  # channel_qubits="same" is an equivalent way to specify channel_qubits=gate_qubits
noise_model.add(gate_name="cx", gate_qubits=[qubits[1], qubits[0]], channel=ad, channel_qubits=[qubits[1]], when="pre")

# # Rule 4.
noise_model.add(gate_name="h", gate_qubits=1, channel=pad, channel_qubits="same")

# # Rule 5.
noise_model.add(gate_name=ANY, gate_qubits=2, channel=d, channel_qubits="same")

# # Rule 6.
noise_model.add(gate_name=ANY, gate_qubits=1, channel=kraus_pd, channel_qubits="same")
noise_model.add(gate_name=ANY, gate_qubits=2, channel=kraus_pd, channel_qubits="same")

# # Rule 7.
noise_model.add_readout_error(qubits[0], 0.3, 0.6)

# # Rule 8.
noise_model.add_readout_error(ANY, 0.1, 0.2)

**Custom Gates**

Additionally, __triggers__ can be defined with custom gates (a `unitary` in Qiskit, or `MatrixGate` in Cirq). If you wish to create custom gates with their own labels, then you should add those labels to the noise model's list of supported gates, just like in Qiskit. To do this, you can use the `add_supported_gate()` method of the `NoiseModel` class. This method takes two arguments: the gate name, and the number of qubits that the gate acts on. For instance, if you have a custom gate called `my_gate` that acts on 2 qubits, you can add it to the noise model using `noise_model.add_supported_gate('my_gate', 2)`.

In [ ]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
qubits = circuit.qubits

# Apply some custom unitaries (which for the purposes of the example are just the identity)
circuit.unitary(np.eye(2), [qubits[0]], label="U_0")
circuit.unitary(np.eye(2), [qubits[1]], label="U_1")
circuit.unitary(np.eye(4), [qubits[0], qubits[1]], label="U_01")

# Make a noise model
noise_model = NoiseModel(qubits=qubits, name="example-qiskit-noise-model", description="An example noise model for 2 qubits")

# Add the unitary names to the list of supported gates
noise_model.add_supported_gate(gate_name="U_0", n_qubits=1)  # Acts on 1 qubit
noise_model.add_supported_gate(gate_name="U_1", n_qubits=1)  # Acts on 1 qubit
noise_model.add_supported_gate(gate_name="U_01", n_qubits=2)  # Acts on 2 qubits

# Add some noise for these gates
d1 = DepolarizingChannel(num_qubits=1, p=0.3)
noise_model.add(gate_name="U_0", gate_qubits=[qubits[0]], channel=d1, channel_qubits="same")
noise_model.add(gate_name="U_1", gate_qubits=[qubits[0]], channel=d1, channel_qubits="same")
noise_model.add(gate_name="U_01", gate_qubits=qubits[0:2], channel=d1, channel_qubits="same")


**Multiple matching triggers**

In many cases, triggers overlap. If a gate in the circuit matches a specific trigger (i.e. __cx__ on qubits __(q0__ __,__ __q1)__), as well as a more general trigger (i.e. __any__ gate on __2__ qubits), only the noise from the more specific trigger is applied. <br><br>

<span style="color:orange">**Note**:</span> This behavior is necessary for the noise model above to match rules **(1) - (8)** <br><br>

Since it is not always obvious which trigger is 'more general', we specifically take the following view: gate triggers are more specific than qubit triggers -- i.e. a trigger that matches a specific gate but generic qubits is considered _more specific_ than a trigger than matches a generic gate but specific qubits. Precisely, the way that noise is applied to a circuit follows the lookup procedure in the flowchart below, which displays the decision process when a gate called \<gate_name\> is encountered in the circuit acting on \<n\> qubits with labels \<qubits\>. <br><br>

noise lookup.svg

**Errors**

When defining a `NoiseChannel`, missing or inconsistent parameters return errors.

In [ ]:
k_wrong = KrausChannel(kraus_ops=[np.array([[0, 0], [0, 0]])*np.sqrt(0.5), np.array([[0, 1], [1, 0]])*np.sqrt(0.5)])

In [ ]:
pauli_wrong = PauliChannel(pauli_probs={"II": 0.5, "XI": 0.25, "YY": 0.2})

When adding noise to the `NoiseModel`, errors are raised whenever things don't add up:

In [ ]:
# H gates only act on single qubits, so this trigger is invaild.
noise_model.add(gate_name="h", gate_qubits=[qubits[0],qubits[1]], channel=pd, channel_qubits=[qubits[0]])

In [ ]:
# d is a DepolarizingNoise instance on 2 qubits, but only one target is given
noise_model.add(gate_name="h", gate_qubits=[qubits[0]], channel=d, channel_qubits=[qubits[0]])

In [ ]:
# Noise is triggered by a single qubit, and attempted to be applied to that same qubit, but the channel is a 2-qubit channel.
noise_model.add(gate_name=ANY, gate_qubits=1, channel=d, channel_qubits="same")

In [ ]:
# Noise is triggered by any gate on any single qubit. To apply a channel to the same qubits, it should be a single-qubit channel, or act on the same number of qubits as the gate (1 again, in this example).
noise_model.add(gate_name=ANY, gate_qubits=1, channel=d, channel_qubits="same")

An error is also raised if you try to add a noise rule for a gate name that is not in the list of supported gates:

In [ ]:
noise_model.add(gate_name="'unsupported'", gate_qubits=qubits[0:2], channel=d, channel_qubits="same")

**Config and validation**

We mentioned earlier that noise should be specified for all gates and readout errors should always be specified for all qubits, otherwise an error will be thrown upon validation (creation of an emulator job). This restriction can be dropped by setting a config option. Specifically, setting the `validate_model` flag of the `noise` config dictionary to `False`:

In [ ]:
noisy_config["noise"]["validate_model"] = False

# 6) Observables

The Fermioniq emulator supports the computation of expectation values of general (local) observables. Observables can be specified via strings of Pauli operators, via:
- Cirq's [`PauliSum`](https://quantumai.google/reference/python/cirq/PauliSum) or [`PauliString`](https://quantumai.google/reference/python/cirq/PauliString) classes (if using Cirq).
- Qiskit's [`SparsePauliOp`](https://qiskit.org/documentation/stubs/qiskit.quantum_info.SparsePauliOp.html) class (if using Qiskit).
  - Note that operators in this class must technically act on all qubits, hence:
   - The number of qubits in the operator must equal the number of `qubits` in the emulator `config`.
   - Qubits are assumed to be in the order specified in the emulator `config`.
  
  To easily specify operators that act on all qubits, it is convenient to use the `from_sparse_list()` constructor of the `SparsePauliOp` class.

The code snippet below shows how to specify observables when using Qiskit.

In [ ]:
from fermioniq.config.defaults import standard_config
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit

def qiskit_bell_pair() -> QuantumCircuit:
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit

# Make a qiskit circuit
circuit = qiskit_bell_pair()

# Make a qiskit pauli operator that is the sum of the following two observables:
# -1 * X(qubit[0]) Z(qubit[1])
# 2 * Y(qubit[0]) Y(qubit[1])
qiskit_pauli = SparsePauliOp.from_sparse_list([("ZX", [1, 0], -1), ("YY", [0, 1], 2)], num_qubits=circuit.num_qubits)

# Make a standard config
config = standard_config(circuit)

# Make an output config with the observable in it and put it into the config (under 'expectation_values')
output_config = {
    "expectation_values": {
        "enabled": True,
        "observables": {"-XZ + 2YY": qiskit_pauli},  # dict of observables and their corresponding names, for each observable, the expectation value will be computed
    }
}
config["output"] = output_config

# Make an emulator job with that output config
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)

rich.print(client.schedule_and_wait(emulator_job))

The code snippet below shows how to specify observables when using Cirq.


Note that, after creating the EmulatorJob instance, both configs essentially look the same, up to qubit labelling.

In [ ]:
import cirq
from cirq.ops.linear_combinations import PauliSum
from cirq.ops.pauli_string import PauliString

# Create a Bell-pair circuit
circuit = cirq.Circuit()
circuit.append(cirq.H(cirq.GridQubit(0, 0)))
circuit.append(cirq.CNOT(cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)))

# The default config uses the default cirq qubit order, obtained as follows
a, b = cirq.QubitOrder.DEFAULT.order_for(circuit.all_qubits())
# Note that the circuit.all_qubits() might return the qubits in a different order!

# Make cirq pauli operator
cirq_pauli = PauliSum.from_pauli_strings(
    [PauliString(-1, cirq.Z(b), cirq.X(a)), PauliString(2, cirq.Y(a), cirq.Y(b))]
)

# Make a standard config
config = standard_config(circuit)

# Make an output config with the observable in it and put it into the config
output_config = {
    "expectation_values": {
        "enabled": True,
        "observables": {"-XZ + 2YY": cirq_pauli},  # dict of observables and their corresponding names, for each observable, the expectation value will be computed
    }
}
config["output"] = output_config

# Make an emulator job with that output config (which serializes the config)
emulator_job = fermioniq.EmulatorJob(circuit=circuit, config=config)

rich.print(client.schedule_and_wait(emulator_job))

# 7) VQE

In this section, we'll explore how to set up and run a [Variational Quantum Eigensolver](https://arxiv.org/pdf/2012.09265.pdf) (VQE) using the emulator.
VQE is a hybrid quantum-classical algorithm designed to minimize the expectation value of an observable for a given parameterized quantum circuit by performing updates on the parameters.

We can run VQE with the emulator in 5 steps: <br>

1) Define parameters to optimize with Qiskit’s [`Parameter`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.Parameter) class.
   The names passed ("parameter 1" and "parameter 2" in the example below) are the names used in the output of the emulator. <br>

2) Use these parameters in the construction of a Qiskit circuit. Parameters can be utilized multiple times and can also be input as linear expressions (e.g. `0.2 * p1 + 1`). Note that more intricate expressions or combinations of multiple parameters for a single gate are not supported. <br>

3) Define a single observable to minimize (see tutorial 6). <br>

4) Enable the optimizer in the config, and add the observable. <br>

5) Submit a job with the defined circuit and config, and wait for optimization.

In [ ]:
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from fermioniq.config.defaults import standard_config

# 1: Define the parameters
p1 = Parameter("parameter 1")
p2 = Parameter("parameter 2")

# 2: Build the parameterized circuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.rx(p1, 0) # Use parameter 1 in the construction of the gate
circuit.rz(0.2*p2, 1) # Use parameter 2 in the construction of the gate

# 3: Define single observable to minimize (as sum over local observables)
observable = SparsePauliOp.from_sparse_list([("ZX", [1, 0], -1), ("YY", [0, 1], 2)], num_qubits=circuit.num_qubits)

# 4: Enable optimization in the config and add the observable
config = standard_config(circuit)
config["optimizer"] = {
"enabled": True,
"observable": {"-XZ + 2YY": observable}
}

# 5: Submit a job and wait for optimization
emulator_job = fermioniq.EmulatorJob(circuit=circuit,config=config)
result = client.schedule_and_wait(emulator_job)

---
**Results**

The results show the observable expection value ("Loss") and fidelity of the emulation for each iteration. The fidelity serves as a measure of the reliability of the loss at each step. To extract the raw optimizer data, we can use the `optimizer_data()` method of the output object.

In [ ]:
# Full results table
rich.print(result)

# Extract optimizer data
print("Optimizer history:", result.optimizer_data(circuit_number=0, run_number=0))

---
**Config Settings**

Here we give an overview of the different settings which can be specified in the "optimizer" section of the config.

- `enabled`: Set to `True` to use the optimizer. <br>
The default is `False`, in which case a normal emulation is performed with parameters set to zero.
- `observable`:  The observable for which the expectation value is minimized using the VQE algorithm. <br>
Note: this can only be one observable.
- `evaluation_mode`: Specifies the evaluation method of the observable expectation value. Currently, the only option is `contract`, which uses exact MPS contraction for evaluation.
- `optimizer_name`: Specifies the optimizer used. Currently, only the [SPSA](https://www.jhuapl.edu/SPSA/PDF-SPSA/Spall_An_Overview.PDF) optimizer is available.
- `optimizer_settings`: A dictionary containing the settings for the classical optimizer. See the options for SPSA in the next paragraph.
- `initial_param_values`: Specifies the initial values for the parameters. If not provided, the optimizer randomly initializes the parameters.
- `initial_param_noise` (default 0.1): This parameter controls the amount of random noise to the initial parameter values.

**SPSA Settings**

The settings that can be set when using SPSA are

- `learning_rate` (default 5): The step size or learning rate for updating the parameters in each iteration. A higher value can lead to faster convergence but may risk overshooting the optimal values.
- `learning_rate_exponent` (default 0.6): Exponent applied to the learning rate in each iteration.
- `perturbation` (default 1e-2): The perturbation or finite difference used to approximate the gradient. A smaller perturbation provides a more accurate gradient estimate but may increase computational cost.
- `perturbation_exponent` (default 0.1): Exponent applied to the perturbation in each iteration.
- `eps` (default 1e-14): A small value to prevent division by zero in the gradient calculation. It ensures numerical stability during optimization.
- `max_iter` (default 100): The maximum number of iterations (updates to the parameters).
- `params_tol` (default 1e-8):  Convergence tolerance for changes in the parameters. If the maximum change in parameters between iterations falls below this threshold, the optimization process stops.
- `fun_tol` (default 1e-7): Convergence tolerance for changes in cost function (in this case observable expectation value).

Below is an example config for the parameterized circuit defined in the section above. If we want to measure the expectation values of any other observables using the circuit with optimized parameters, we can list them in the `output` section of the config like we did in tutorial 6.

In [ ]:
from fermioniq.config.defaults import standard_config

config = standard_config(circuit)

config["optimizer"] = {
    "enabled": True,
    "evaluation_mode": "contract",
    "optimizer_name": "spsa",
    "optimizer_settings": {
      "learning_rate": 5,
      "fun_tol": 1e-8,
      "max_iter": 100,
     },

    # Put the observable to minimize in the optimizer config
    "observable": {"-XZ + 2YY": observable},
    }

# (optional) Put any other observables we would want to measure in the regular config
config["output"] = {
    "expectation_values": {
        "enabled": True,
        "observables": {"ZZ": SparsePauliOp("ZZ", 1.0)},
    }
}

rich.print(config)

# 8) Asynchronous jobs

Sometimes it is not desirable for the program to block and wait for the emulation to finish, especially for larger jobs. It is possible to submit a job and then continue with program execution, and later on retrieve the results of the job.

Let's create a qiskit circuit as usual.

In [ ]:
from qiskit import QuantumCircuit

def qiskit_bell_pair():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit

circuit = qiskit_bell_pair()

# Create an emulator job
emulator_job = fermioniq.EmulatorJob(circuit=circuit)

We can schedule an asynchronous job using the `schedule_async()` method of the client.

In [ ]:
# Asynchronously schedule the job
job = client.schedule_async(emulator_job)

# Print the job ID and status
print(f"Job id {job.id}. Status {job.status_code}: {job.status}.")

At any time we can block and wait for the emulation to finish, and optionally recieve live updates on the progress. This can be done using the `subscribe_to_events()` method of the client.

In [ ]:
# Block and wait for a job to finish. Here we pass the job id to the subscribe_to_events( ) method
client.subscribe_to_events(job.id, log_updates=True)

Alternatively, we can check whether the job has finished at a later time. If it has, we can retrieve the results.

In [ ]:
status = client.get_status(job.id)

if status == 'finished':
    # Retrieve and print results
    rich.print(client.get_results(job.id))

If you lose the job ID or just want to view the status of your jobs, you can list them using the `jobs()` method of the client. Printing the jobs object with rich.print renders a table. To get the jobs as a list, use the `job_list` attribute.

In [ ]:
# Limit: (maximum) N=number of jobs to retrieve
# Offset: From which point to list jobs (leave at 0 to list the most recent jobs)
jobs = client.jobs(limit=10, offset=0)
rich.print(jobs)

# Print the results of the most recent job
last_result = client.get_results(jobs.job_list[0].id)
rich.print(last_result)

Finally, it is possible to cancel a job by using the `cancel()` method of the client.

In [ ]:
# Schedule a job and immediately cancel it
job = client.schedule_async(emulator_job)
client.cancel(job.id)

# 9) Batch jobs

It is possible to submit multiple circuits simultaneously. How this works in practice is explained below.

Afterwards, we'll discuss a benchmarking method called 'Schrödingers Microscope' that makes use of batch jobs.

Running a batch job is as simple as providing a list of circuits to EmulatorJob.

If a single config is provided, it will be used for all circuits in the list. Alternatively, a separate config can be provided for every individual circuit (in which case the list of circuits and the list of configs should be the same length).

For large batches of smaller jobs (say, more than 100), we have a specialized remote config backend for the task: 'cpu-batch'. Try using this to see if it helps to speed up batch jobs. **CURRENTLY UNAVAILABLE -- will be added back in shortly.**

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

def qiskit_bell_pair():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit

# Create 10 qiskit circuits
all_circuits = [qiskit_bell_pair() for _ in range(10)]

# Create a single config
config = standard_config(all_circuits[0])

# Run all circuits using the same config
emulator_job = fermioniq.EmulatorJob(circuit=all_circuits, config=config)
# (Alternatively, a list of configs can be provided as the config keyword argument)

# Run batch job
output = client.schedule_and_wait(emulator_job)
rich.print(output)

When running a batch job, the output is a list of outputs, each containing a `circuit_number` corresponding to the circuit that was emulated.

When used in combination with the dmrg `init_mode` = `lowerD`, then each output will also have a `run_number` corresponding to the bond dimension in the list of bond dimensions used (see Tutorial 3 for details).

---
**Schrödinger's Microscope**

For this tutorial we'll play around with some slightly larger circuits, and with sending batch jobs to the emulator. We'll use some benchmark circuits from [_"Scalable Benchmarks for Gate-Based Quantum Computers"_](https://arxiv.org/abs/2104.10698) by Arjan Cornelissen, Johannes Bausch, and András Gilyén.

Full details can be found in the paper, but briefly: the benchmark circuit work by preparing quantum states that encode complex numbers from the Riemann sphere $\mathbb{C} \cup \{\infty\}$, and then performs various Mobius transformations on them. The qubits can then be measured and the resulting probability distributions compared to the ideal ones, which can be computed analytically. The probability distributions can also be plotted, and visually compared, which gives a nice way to percieve the difference in performance between different pieces of quantum hardware, or -- in our case -- **quantum emulators**.

The first circuit we'll consider is "_Schrödinger's Microscope_". This performs, recursively, the following transformation:

$F : z \mapsto \frac{z^2 + i}{iz^2 + 1}$.

So the '$n$th level' of Schrödinger's Microscope is the transformation

$F^{\circ n} = F \circ \cdots \circ F$.

We encode complex numbers inside quantum states as

$|\psi_z\rangle = \frac{z|0\rangle + |1\rangle}{\sqrt{|z|^2 + 1}}$, and $|\psi_\infty\rangle = |0\rangle$.

The $n$ th level can be implemented by the quantum circuit below, which takes as input a complex number $z$ (because it needs to encode this as $|\psi_z\rangle$), and a level.


In [ ]:
def schrodingers_microscope(z: complex, level=2):
    """
    Schrödinger's Microscope circuit, from https://arxiv.org/abs/2104.10698
    """

    def _prep(circuit: QuantumCircuit, z: complex):
        for i in range(len(circuit.qubits)):
            prep_unitary = (1 / np.sqrt(np.abs(z) ** 2 + 1)) * np.array(
                [[z, 1], [1, -np.conj(z)]]
            )
            circuit.unitary(prep_unitary, [i], label="|enc(z)>")

    def _mobius(circuit: QuantumCircuit, qa: int, qb: int):
        circuit.cx(qa, qb)
        circuit.s(qa)
        circuit.h(qa)
        circuit.s(qa)

    n_qubits = 2**level  # No. qubits is 2^level
    circuit = QuantumCircuit(n_qubits)

    _prep(circuit, z)  # Prepare the intial state

    for l in range(level):
        step = 2**l
        for i in range(0, n_qubits, 2 * step):
            _mobius(circuit, i, i + step)

    return circuit

In [ ]:
import numpy as np
z = 0.5 + 0.5j
level = 2

circuit = schrodingers_microscope(z, level=level)

print(circuit)

The probability of interest for us will be the quantity

$p^{(n)}(z) = \frac{1}{|F^{\circ n}|^2 + 1}$.

We will plot the distribution over the complex plane. For example, below we do this (using analytically computed values) with the real and imaginary parts ranging from -1.5 to 1.5.

In [ ]:
def get_correct_probabilities(
    min_val: int, max_val: int, n_pixels: int, level: int = 2
) -> np.ndarray:
  """Given a minimum and maximum value for the real and imaginary parts, and a number
  of pixels, returns 2d array containing the correct probabilities for each complex number.

  This constitutes the 'target'.

  Parameters
  ----------
  min_val
    Minimum value
  max_val
    Maximum value
  n_pixels
    Number of pixels in image
  level
    The level of Schrodinger's microscope to implement (see https://arxiv.org/abs/2104.10698).

  Returns
  -------
  probs
    Array of true probabilities.
  """

  def f(_z, level):
    if level > 1:
      _z = f(_z, level - 1)
    return (_z**2 + 1j) / (1j * _z**2 + 1)

  x_range = y_range = np.linspace(min_val, max_val, n_pixels)

  probs = np.zeros((len(x_range), len(y_range)))
  for x, z_real in enumerate(x_range):
    for y, z_imag in enumerate(y_range):
      z = z_real + 1.0j * z_imag
      probs[x, y] = 1 / (np.abs(f(z, level)) ** 2 + 1)
  return probs


def plot_probabilities(probs: np.ndarray):
    import matplotlib.pyplot as plt

    x = np.arange(probs.shape[0])
    y = np.arange(probs.shape[1])
    plt.pcolormesh(x, y, probs, shading="auto", cmap=plt.cm.Greys)
    plt.show()

In [ ]:
probs = get_correct_probabilities(-1.5, 1.5, n_pixels=500, level=level)
plot_probabilities(probs)

The probability $p^{(n)}(z)$ is also given by the probability of measuring $|1\rangle$ on the first qubit in the circuit, conditioned on all other qubits being in the all-zero state (read the paper for why :) ).

We can obtain the same probability distribution by running the quantum circuit, and estimating this probability for each point on the complex plane. This can be done via emulation, or by running the circuit on a real quantum device (or both).

We'll run the circuit on the fermioniq emulator. For convenience, we made a utility function `generate_schrodingers_microscope_circuits()` that will generate all the circuits (i.e. for all points on the complex plane) for you.

In [ ]:
def generate_schrodingers_microscope_circuits(
    min_val: int, max_val: int, n_pixels: int, level: int = 2
) -> list[QuantumCircuit]:
  """Given a minimum and maximum value for the real and imaginary parts, and a number
  of pixels, returns a list of circuits (one for each pixel).

  Parameters
  ----------
  min_val
    Minimum value
  max_val
    Maximum value
  n_pixels
    Number of pixels in image
  level
    The level of Schrodinger's microscope to implement (see https://arxiv.org/abs/2104.10698).

  Returns
  -------
  circuits
    List of circuits for each pixel / point in the complex plane.
  """
  x_range = y_range = np.linspace(min_val, max_val, n_pixels)

  circuits = []
  for z_real in x_range:
    for z_imag in y_range:
      z = z_real + 1.0j * z_imag
      circuit = schrodingers_microscope(z, level=level)
      circuits.append(circuit)
  return circuits

In [ ]:
n_pixels = 7  # We recommend to set to at most 40

# Each pixels is a circuit, so 20 x 20 pixels is already 400 circuits
all_circuits = generate_schrodingers_microscope_circuits(-1.5, 1.5, n_pixels=n_pixels, level=level)

# Save this for later
n_qubits = len(all_circuits[0].qubits)

Now we can run the emulation. Note that it might take a little time to send all circuits across, and then to retrieve the results!

To make things run a little faster, we'll use the 'cpu-batch' remote config backend.

In [ ]:
# We'll override the output config for this emulation
# We don't need any samples, but we do want to compute the amplitudes on the basis states
# 10...0 and 00...0
state_00__0 = "0" * n_qubits
state_10__0 = "1" + "0" * (n_qubits-1)
basis_states = [state_00__0, state_10__0]

# Generate config
config = standard_config(all_circuits[0])

output_config_update = {
    "amplitudes": {
        "enabled": True,
        "bitstrings": basis_states,
    },
    "sampling": {
      "enabled": False
    }
}

dmrg_config_update = {
    "D": 2,
}
config["output"].update(output_config_update)
config["dmrg"].update(dmrg_config_update)

emulator_job = fermioniq.EmulatorJob(circuit=all_circuits, config=config)

output = client.schedule_and_wait(emulator_job)

We can compute the quantity $p^{(n)}(z)$ by computing the probability of measuring $|1\rangle$ on the first qubit conditioned on all other qubits having been measured as $|0\rangle$. In other words,

$p^{(n)}(z) = \frac{|\alpha_{10\dots 0}|^2}{|\alpha_{00\dots 0}|^2 + |\alpha_{10\dots 0}|^2}$,

where $\alpha_{10\dots 0}$ and $\alpha_{00\dots 0}$ are the amplitudes on the state $|10\dots 0\rangle$ and $|00\dots 0\rangle$, respectively.

We can compute this quantity for every circuit, which corresponds to a particular choice of $z$.

In [ ]:
import numpy as np

all_amplitudes = [output.amplitudes(circuit_number=i, run_number=0) for i in range(len(all_circuits))]

all_probs = []
for amplitudes in all_amplitudes:
    amp_0__00 = amplitudes[state_00__0]
    amp_0__01 = amplitudes[state_10__0]

    p = np.abs(amp_0__01)**2 / (np.abs(amp_0__00)**2 + np.abs(amp_0__01)**2)

    all_probs.append(p)

# Reshape the probabilities back into a grid
grid_probs = np.array(all_probs).reshape((n_pixels, n_pixels))

plot_probabilities(grid_probs)

The image looks nothing like what we expect. Try to increase the resolution (n_pixels) up to 40 to get a better image. Can you find out what setting needs to be changed to get an image that corresponds with the analytical result? (Something with entanglement...)